#### Imports

In [122]:
import pandas as pd
import hashlib as hl
import base64
import zlib
from cryptography.fernet import Fernet


#### Work

In [132]:
subgroups = read_csv('subgroups')
subgroups

,id,name
0,tot,Total Population
1,anh,Asian Non-Hispanic
2,bnh,Black Non-Hispanic
3,wnh,White Non-Hispanic
4,hsp,Hispanic


In [7]:
categories = read_csv('categories')
categories

,id,name
0,demo,Demographic Conditions
1,econ,Household Economic Security
2,hsaq,"Housing Security, Affordability, and Quality"
3,hopd,Housing Production
4,qlao,Quality of Life and Access to Opportunity


In [131]:
indicators = read_csv('indicators')
indicators["id"] = indicators["name"].apply(encode_and_trim)
indicators.head(20)

,name,category.id,category.name,type.id,hasPercent,hasRatePer100k,hasRacialBreakdown,id
0,Mutually Exclusive Race/Hispanic Origin,demo,Demographic Conditions,SAMPLE,True,False,True,4b07ea
1,Age,demo,Demographic Conditions,SAMPLE,True,False,True,ff9f1f
2,Foreign-born Population,demo,Demographic Conditions,SAMPLE,True,False,True,d6f602
3,Limited English Speaking Population,demo,Demographic Conditions,SAMPLE,True,False,True,1a482b
4,Educational Attainment (Highest Grade Completed),econ,Household Economic Security,SAMPLE,True,False,True,be96d9
5,Median Household Income (2019 dollars),econ,Household Economic Security,SAMPLE,False,False,True,ffc58d
6,Area Median Income (AMI),econ,Household Economic Security,SAMPLE,True,False,True,5f97f4
7,Labor Force,econ,Household Economic Security,SAMPLE,True,False,True,b43183
8,Occupation,econ,Household Economic Security,SAMPLE,True,False,True,5ea1f8
9,Median Wages by Occupation (2019 dollars),econ,Household Economic Security,SAMPLE,False,False,True,5db4b8


In [ ]:
indicators

In [134]:
tables = read_csv('tables')
tables = tables.merge(subgroups.rename(columns={"name": "subgroup", "id": "subgroup_id"}), on="subgroup")
tables

,category,indicator,subgroup,name,subgroup_id
0,Demographic Conditions,Mutually Exclusive Race/Hispanic Origin,Total Population,Total Population - Mutually Exclusive Race/His...,tot
1,Demographic Conditions,Age,Total Population,Total Population - Age,tot
2,Demographic Conditions,Foreign-born Population,Total Population,Total Population - Foreign-born Population,tot
3,Demographic Conditions,Limited English Speaking Population,Total Population,Total Population - Limited English Speaking Po...,tot
4,Household Economic Security,Educational Attainment (Highest Grade Completed),Total Population,Total Population - Educational Attainment (Hig...,tot
...,...,...,...,...,...
124,Quality of Life and Access to Opportunity,Education Access - Access to Broadband Interne...,Hispanic,Hispanic - Education Access - Access to Broadb...,hsp
125,Quality of Life and Access to Opportunity,Educational Outcomes - English Language Arts (...,Hispanic,Hispanic - Educational Outcomes - English Lang...,hsp
126,Quality of Life and Access to Opportunity,Educational Outcomes - Math Test Proficiency,Hispanic,Hispanic - Educational Outcomes - Math Test Pr...,hsp
127,Quality of Life and Access to Opportunity,Educational Outcomes - High School Graduation ...,Hispanic,Hispanic - Educational Outcomes - High School ...,hsp


In [26]:
test = tables.set_index(["category","subgroup","indicator"])
test

name
category                                     subgroup           indicator                                                                                 
Demographic Conditions                       Total Population   Mutually Exclusive Race/Hispanic Origin  Total Population - Mutually Exclusive Race/His...
                                                                Age                                                                 Total Population - Age
                                                                Foreign-born Population                         Total Population - Foreign-born Population
                                                                Limited English Speaking Population      Total Population - Limited English Speaking Po...
                                             Asian Non-Hispanic Mutually Exclusive Race/Hispanic Origin  Asian Non-Hispanic - Mutually Exclusive Race/H...
...                                                                                                                                                    ...
Housing Security, Affordability, and Quality Total Population   Population in NYCHA Housing                 Total Population - Population in NYCHA Housing
                                             Asian Non-Hispanic Population in NYCHA Housing               Asian Non-Hispanic - Population in NYCHA Housing
                                             Black Non-Hispanic Population in NYCHA Housing               Black Non-Hispanic - Population in NYCHA Housing
                                             White Non-Hispanic Population in NYCHA Housing               White Non-Hispanic - Population in NYCHA Housing
                                             Hispanic           Population in NYCHA Housing                         Hispanic - Population in NYCHA Housing

[129 rows x 1 columns]

In [147]:
# test.loc[("Demographic Conditions","Asian Non-Hispanic")]

# test.loc["Demographic Conditions","Asian Non-Hispanic"]
# test.xs("Total Population", level="subgroup")
test.unstack()


name  \
indicator                                                                           Access to Jobs   
category                                     subgroup                                                
Demographic Conditions                       Asian Non-Hispanic                                NaN   
                                             Black Non-Hispanic                                NaN   
                                             Hispanic                                          NaN   
                                             Total Population                                  NaN   
                                             White Non-Hispanic                                NaN   
Household Economic Security                  Asian Non-Hispanic                                NaN   
                                             Black Non-Hispanic                                NaN   
                                             Hispanic                                          NaN   
                                             Total Population                                  NaN   
                                             White Non-Hispanic                                NaN   
Housing Production                           Total Population                                  NaN   
Housing Security, Affordability, and Quality Asian Non-Hispanic                                NaN   
                                             Black Non-Hispanic                                NaN   
                                             Hispanic                                          NaN   
                                             Total Population                                  NaN   
                                             White Non-Hispanic                                NaN   
Quality of Life and Access to Opportunity    Asian Non-Hispanic                                NaN   
                                             Black Non-Hispanic                                NaN   
                                             Hispanic                                          NaN   
                                             Total Population    Total Population - Access to Jobs   
                                             White Non-Hispanic                                NaN   

                                                                                                       \
indicator                                                                           Access to Transit   
category                                     subgroup                                                   
Demographic Conditions                       Asian Non-Hispanic                                   NaN   
                                             Black Non-Hispanic                                   NaN   
                                             Hispanic                                             NaN   
                                             Total Population                                     NaN   
                                             White Non-Hispanic                                   NaN   
Household Economic Security                  Asian Non-Hispanic                                   NaN   
                                             Black Non-Hispanic                                   NaN   
                                             Hispanic                                             NaN   
                                             Total Population                                     NaN   
                                             White Non-Hispanic                                   NaN   
Housing Production                           Total Population                                     NaN   
Housing Security, Affordability, and Quality Asian Non-Hispanic                                   NaN   
                                             Black Non-Hispanic       

In [136]:
test.head(25)

name
category                    subgroup           indicator                                                                                          
Demographic Conditions      Total Population   Mutually Exclusive Race/Hispanic Origin           Total Population - Mutually Exclusive Race/His...
                                               Age                                                                          Total Population - Age
                                               Foreign-born Population                                  Total Population - Foreign-born Population
                                               Limited English Speaking Population               Total Population - Limited English Speaking Po...
                            Asian Non-Hispanic Mutually Exclusive Race/Hispanic Origin           Asian Non-Hispanic - Mutually Exclusive Race/H...
                                               Age                                                                        Asian Non-Hispanic - Age
                                               Foreign-born Population                                Asian Non-Hispanic - Foreign-born Population
                                               Limited English Speaking Population               Asian Non-Hispanic - Limited English Speaking ...
                            Black Non-Hispanic Mutually Exclusive Race/Hispanic Origin           Black Non-Hispanic - Mutually Exclusive Race/H...
                                               Age                                                                        Black Non-Hispanic - Age
                                               Foreign-born Population                                Black Non-Hispanic - Foreign-born Population
                                               Limited English Speaking Population               Black Non-Hispanic - Limited English Speaking ...
                            White Non-Hispanic Mutually Exclusive Race/Hispanic Origin           White Non-Hispanic - Mutually Exclusive Race/H...
                                               Age                                                                        White Non-Hispanic - Age
                                               Foreign-born Population                                White Non-Hispanic - Foreign-born Population
                                               Limited English Speaking Population               White Non-Hispanic - Limited English Speaking ...
                            Hispanic           Mutually Exclusive Race/Hispanic Origin           Hispanic - Mutually Exclusive Race/Hispanic Or...
                                               Age                                                                                  Hispanic - Age
                                               Foreign-born Population                                          Hispanic - Foreign-born Population
                                               Limited English Speaking Population                  Hispanic - Limited English Speaking Population
Household Economic Security Total Population   Educational Attainment (Highest Grade Completed)  Total Population - Educational Attainment (Hig...
                                               Median Household Income (2019 dollars)            Total Population - Median Household Income (20...
                                               Area Median Income (AMI)                                Total Population - Area Median Income (AMI)
                                               Labor Force                                                          Total Population - Labor Force
                                               Occupation                                                            Total Population - Occupation

In [138]:
test.loc["Demographic Conditions","Asian Non-Hispanic","Age"]["name"]

'Asian Non-Hispanic - Age'

#### Utils

In [6]:
def read_csv(name):
  return pd.read_csv('../data/'+name+".csv")

In [106]:
foo = hl.sha1(b'Moderate-income (81-120% AMI) Households').hexdigest()[:6]
foo

'6a10aa'

In [118]:
# data = foo
data = "Moderate-income (81-120% AMI) Households"
encodedBytes = base64.b64encode(data.encode("utf-8"))
# encodedBytes = base64.b16encode(data.encode("utf-8"))
encodedStr = str(encodedBytes, "utf-8")

print(encodedStr)

TW9kZXJhdGUtaW5jb21lICg4MS0xMjAlIEFNSSkgSG91c2Vob2xkcw==


In [69]:
data.encode("utf-8")

b'Demographic Conditions'

In [129]:
def encode_and_trim(val):
  return hl.sha1(val.encode("utf-8")).hexdigest()[:6]
  hash = hl.sha1(val.encode("utf-8")).digest()
  encodedBytes = base64.b16encode(hash)[:6]
  return str(encodedBytes, "utf-8")

In [130]:
encode_and_trim("Moderate-income (81-120% AMI) Households")

'6a10aa'

In [128]:
compressed = zlib.compress(b'Moderate income')
# len(compressed)
encodedBytes = base64.b64encode(compressed)
# encodedBytes = base64.b16encode(data.encode("utf-8"))
encodedStr = str(encodedBytes, "utf-8")
encodedStr

'eJzzzU9JLUosSVXIzEvOz00FAC30Bc0='

In [114]:
len("Moderate-income (81-120% AMI) Households".encode("utf-8"))

40

In [127]:
f = Fernet(Fernet.generate_key())
token = f.encrypt("Moderate-income (81-120% AMI) Households".encode("utf-8"))
token

b'gAAAAABiIT0795DnerKZzQIst0gBX1lo2kQnrpXrK6cpRBVWbyd42npBU96nRjOJqTMWaKxlZZ1gnrxVF2uTd8l5Om0bottYPmirhSDqxGW7Q5rkDLSE_2bw5WPXx2858kHFlbg1OThY'